# Exercise 2: Decision Trees

In this assignment you will implement a Decision Tree algorithm as learned in class.

## Read the following instructions carefully:

1. This jupyter notebook contains all the step by step instructions needed for this exercise.
1. Submission includes this notebook only with the exercise number and your ID as the filename. For example: `hw2_123456789_987654321.ipynb` if you submitted in pairs and `hw2_123456789.ipynb` if you submitted the exercise alone.
1. Write **efficient vectorized** code whenever possible. Some calculations in this exercise take several minutes when implemented efficiently, and might take much longer otherwise. Unnecessary loops will result in point deduction.
1. You are responsible for the correctness of your code and should add as many tests as you see fit. Tests will not be graded nor checked.
1. Write your functions in this notebook only. **Do not create Python modules and import them**.
1. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/) and [numpy](https://www.numpy.org/devdocs/reference/) only. **Do not import anything else.**
1. Your code must run without errors. Make sure your `numpy` version is at least 1.15.4 and that you are using at least python 3.6. Changes of the configuration we provided are at your own risk. Any code that cannot run will not be graded.
1. Write your own code. Cheating will not be tolerated.
1. Answers to qualitative questions should be written in **markdown** cells (with $\LaTeX$ support). Answers that will be written in commented code blocks will not be checked.

## In this exercise you will perform the following:
1. Practice OOP in python.
2. Implement two impurity measures: Gini and Entropy.
3. Construct a decision tree algorithm.
4. Prune the tree to achieve better results.
5. Visualize your results.

# I have read and understood the instructions: 323081950, 227367455

In [138]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# make matplotlib figures appear inline in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Warmup - OOP in python

Our desicion tree will be implemented using a dedicated python class. Python classes are very similar to classes in Java.


You can use the following [site](https://jeffknupp.com/blog/2014/06/18/improve-your-python-python-classes-and-object-oriented-programming/) to learn about classes in python.

In [139]:
class Node(object):
    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, node):
        self.children.append(node)

In [140]:
n = Node(5)
p = Node(6)
q = Node(7)
n.add_child(p)
n.add_child(q)
n.children

[<__main__.Node at 0x1d0af891e20>, <__main__.Node at 0x1d0af50c8b0>]

## Data preprocessing

For the following exercise, we will use a dataset containing mushroom data `agaricus-lepiota.csv`. 

This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family. Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous
one (=there are only two classes **edible** and **poisonous**). 
    
The dataset contains 8124 observations with 22 features:
1. cap-shape: bell=b,conical=c,convex=x,flat=f,knobbed=k,sunken=s
2. cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
3. cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y
4. bruises: bruises=t,no=f
5. odor: almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
6. gill-attachment: attached=a,descending=d,free=f,notched=n
7. gill-spacing: close=c,crowded=w,distant=d
8. gill-size: broad=b,narrow=n
9. gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g,green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
10. stalk-shape: enlarging=e,tapering=t
11. stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r
12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
14. stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
15. stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
16. veil-type: partial=p,universal=u
17. veil-color: brown=n,orange=o,white=w,yellow=y
18. ring-number: none=n,one=o,two=t
19. ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z
20. spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y
21. population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y
22. habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d

First, we will read and explore the data using pandas and the `.read_csv` method. Pandas is an open source library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

In [141]:
# load dataset
data = pd.read_csv('agaricus-lepiota.csv')

One of the advantages of the Decision Tree algorithm is that almost no preprocessing is required. However, finding missing values is always required.

In [142]:
#############################################################################
# TODO: Find the columns with missing values and remove them from the data.#
#############################################################################
for column in data:
    if data[column].isnull().values.any():
        data.drop(labels=[column], axis=1)
#############################################################################
#                             END OF YOUR CODE                              #
#############################################################################

We will split the dataset to `Training` and `Testing` datasets.

In [143]:
from sklearn.model_selection import train_test_split
# Making sure the last column will hold the labels
X, y = data.drop('class', axis=1), data['class']
X = np.column_stack([X,y])
# split dataset using random_state to get the same split each time
X_train, X_test = train_test_split(X, random_state=99)

print("Training dataset shape: ", X_train.shape)
print("Testing dataset shape: ", X_test.shape)

Training dataset shape:  (6093, 22)
Testing dataset shape:  (2031, 22)


In [144]:
data

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,o,o,p,o,o,p,b,c,l,e
8120,x,s,n,f,n,a,c,b,y,e,...,o,o,p,n,o,p,b,v,l,e
8121,f,s,n,f,n,a,c,b,n,e,...,o,o,p,o,o,p,b,c,l,e
8122,k,y,n,f,y,f,c,n,b,t,...,w,w,p,w,o,e,w,v,l,p


In [145]:
np.unique(X_train[:,3])

array(['f', 't'], dtype=object)

In [146]:
[X_train[np.where(X_train[:,-1] == lable)] for lable in np.unique(X_train[:,-1])]


[array([['x', 's', 'y', ..., 'n', 'g', 'e'],
        ['k', 'y', 'e', ..., 'v', 'l', 'e'],
        ['f', 'y', 'n', ..., 's', 'p', 'e'],
        ...,
        ['x', 'f', 'n', ..., 'v', 'd', 'e'],
        ['f', 'y', 'g', ..., 'y', 'd', 'e'],
        ['k', 's', 'e', ..., 'v', 'l', 'e']], dtype=object),
 array([['f', 'y', 'n', ..., 'v', 'p', 'p'],
        ['f', 'f', 'y', ..., 'v', 'p', 'p'],
        ['k', 's', 'n', ..., 'v', 'l', 'p'],
        ...,
        ['x', 'y', 'e', ..., 'v', 'p', 'p'],
        ['x', 'y', 'n', ..., 'v', 'l', 'p'],
        ['f', 'y', 'g', ..., 'y', 'p', 'p']], dtype=object)]

In [147]:
X_train

array([['x', 's', 'y', ..., 'n', 'g', 'e'],
       ['k', 'y', 'e', ..., 'v', 'l', 'e'],
       ['f', 'y', 'n', ..., 'v', 'p', 'p'],
       ...,
       ['x', 'y', 'n', ..., 'v', 'l', 'p'],
       ['k', 's', 'e', ..., 'v', 'l', 'e'],
       ['f', 'y', 'g', ..., 'y', 'p', 'p']], dtype=object)

In [148]:
X_train[np.where(X_train[:,-1] == 'p')]

array([['f', 'y', 'n', ..., 'v', 'p', 'p'],
       ['f', 'f', 'y', ..., 'v', 'p', 'p'],
       ['k', 's', 'n', ..., 'v', 'l', 'p'],
       ...,
       ['x', 'y', 'e', ..., 'v', 'p', 'p'],
       ['x', 'y', 'n', ..., 'v', 'l', 'p'],
       ['f', 'y', 'g', ..., 'y', 'p', 'p']], dtype=object)

## Impurity Measures

Impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. Implement the functions `calc_gini` and `calc_entropy`. You are encouraged to test your implementation (10 points).

In [149]:
def calc_gini(data):
    """
    Calculate gini impurity measure of a dataset.
 
    Input:
    - data: any dataset where the last column holds the labels.
 
    Returns the gini impurity.    
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    space_size = data.shape[0]
    # np.unique returns tuple of elements (index=0) with mapping to their count (index=1) 
    gini = np.square(np.unique(data[:,-1], return_counts=True)[1] / space_size)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return 1.0 - np.sum(gini)

In [150]:
def calc_entropy(data):
    """
    Calculate the entropy of a dataset.

    Input:
    - data: any dataset where the last column holds the labels.

    Returns the entropy of the dataset.    
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    space_size = data.shape[0]
    probabilities = np.unique(data[:,-1], return_counts=True)[1] / space_size
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return -np.sum(probabilities * np.log2(probabilities))

In [151]:
##### Your Tests Here #####
equal_distribution = np.ones(100).reshape(10,10)
unequal_distribution = np.arange(100).reshape(10,10)

print("gini,entropy")
print("Equal distributions:")
print(calc_gini(equal_distribution), calc_entropy(equal_distribution))

print("Unequal distributions:")
print(calc_gini(unequal_distribution), calc_entropy(unequal_distribution))

print("Dataset:")
print(calc_gini(X), calc_entropy(X))

gini,entropy
Equal distributions:
0.0 -0.0
Unequal distributions:
0.8999999999999999 3.321928094887362
Dataset:
0.4995636322379775 0.9993703627906085


## Goodness of Split

Given a feature the Goodnees of Split measures the reduction in the impurity if we split the data according to the feature.
$$
\Delta\varphi(S, A) = \varphi(S) - \sum_{v\in Values(A)} \frac{|S_v|}{|S|}\varphi(S_v)
$$

In our implementation the goodness_of_split function will return either the Goodness of Split or the Gain Ratio as learned in class. You'll control the return value with the `gain_ratio` parameter. If this parameter will set to False (the default value) it will return the regular Goodness of Split. If it will set to True it will return the Gain Ratio.
$$
GainRatio(S,A)=\frac{InformationGain(S,A)}{SplitInformation(S,A)}
$$
Where:
$$
InformationGain(S,A)=Goodness\ of\ Split\ calculated\ with\ Entropy\ as\ the\ Impurity\ function \\
SplitInformation(S,A)=- \sum_{a\in A} \frac{|S_a|}{|S|}\log\frac{|S_a|}{|S|}
$$
NOTE: you can add more parameters to the function and you can also add more returning variables (The given parameters and the given returning variable should not be touch). (10 Points)

In [152]:
def goodness_of_split(data, feature, impurity_func, gain_ratio=False):
    """
    Calculate the goodness of split of a dataset given a feature and impurity function.

    Input:
    - data: any dataset where the last column holds the labels.
    - feature: the feature index.
    - impurity func: a function that calculates the impurity.
    - gain_ratio: goodness of split or gain ratio flag.

    Returns the goodness of split (or the Gain Ration).  
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    if gain_ratio:
        goodness, split_feature_dict = goodness_of_split(data, feature, impurity_func, gain_ratio=False)
        goodness /= impurity_func(data)
    else:
        space_size = data.shape[0]
        feature_values =  np.unique(data[:,feature])
        split_feature_dict = {value : data[np.where(data[:,feature] == value)] for value in feature_values}
        goodness = impurity_func(data) - (sum(len(split_feature_dict[value]) * impurity_func(split_feature_dict[value]) 
                                                             for value in feature_values) / space_size)
        
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return goodness, split_feature_dict 

In [153]:
def best_feature(data, impurity_func, gain_ratio=False):
    goodness = -1
    split_feature_dict = {}
    best_feature = -1
    for feature in range(data.shape[1] - 1):
        tmp_goodness, tmp_split_feature_dict = goodness_of_split(data, feature, impurity_func, gain_ratio=gain_ratio)
        if tmp_goodness > goodness:
            goodness = tmp_goodness
            split_feature_dict = tmp_split_feature_dict
            best_feature = feature
    return best_feature, goodness, split_feature_dict

## Building a Decision Tree

Use a Python class to construct the decision tree. Your class should support the following functionality:

1. Initiating a node for a decision tree. You will need to use several class methods and class attributes and you are free to use them as you see fit. We recommend that every node will hold the feature and value used for the split and its children.
2. Your code should support both Gini and Entropy as impurity measures. 
3. The provided data includes categorical data. In this exercise, when splitting a node create the number of children needed according to the attribute unique values.

Complete the class `DecisionNode`. The structure of this class is entirely up to you. 

Complete the function `build_tree`. This function should get the training dataset and the impurity as inputs, initiate a root for the decision tree and construct the tree according to the procedure you learned in class. (30 points)

In [154]:
class DecisionNode:
    """
    This class will hold everything you require to construct a decision tree.
    The structure of this class is up to you. However, you need to support basic 
    functionality as described above. It is highly recommended that you 
    first read and understand the entire exercise before diving into this class.
    """
    def __init__(self, feature, feature_value, labels:dict):
        self.feature = feature # column index of criteria being tested
        self.feature_value = feature_value
        self.labels = labels
        self.children = []
        self.parent = None
        
    def add_child(self, node):
        if not node:
            return
        self.children.append(node)
        node.parent = self
        
    @staticmethod 
    def get_label(data):
        tag, count = np.unique(data[:,-1], return_counts=True)
        return {tag : count for tag, count in zip(tag, count)}
        
    def __str__(self):
        if not self.parent:
            return f"[ROOT, feature={self.feature}]"
        elif not self.children:
            return f"[{self.parent.feature}={self.feature_value}, leaf]: [{self.labels}]"
        else:
            return f"[{self.parent.feature}={self.feature_value}, feature={self.feature}]: [{self.labels}]"

    def recursive_print(self, tab="\t", depth=0):
        print(tab * depth, self)
        for child in self.children:
            child.recursive_print(tab, depth + 1)
        

In [158]:
def build_tree(data, impurity, gain_ratio=False, chi=1, max_depth=500, feature_value=None):
    """
    Build a tree using the given impurity measure and training dataset. 
    You are required to fully grow the tree until all leaves are pure. 

    Input:
    - data: the training dataset.
    - impurity: the chosen impurity measure. Notice that you can send a function
                as an argument in python.
    - gain_ratio: goodness of split or gain ratio flag
    - chi: chi square p-value cut off (1 means no pruning)
    - max_depth: the allowable depth of the tree

    Output: the root node of the tree.
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    feature, goodness, split_feature_dict = best_feature(data, impurity, gain_ratio=gain_ratio)
    #print(split_feature_dict)
    #print(feature, goodness)
    if not max_depth + 1:
        return None
    
    if feature_value:
        root = DecisionNode(feature, feature_value, DecisionNode.get_label(data))
    else:
        root = DecisionNode(feature, None, None)
        
    if not goodness:
        return root
    
    for value in split_feature_dict:
        split_root = build_tree(split_feature_dict[value], impurity, gain_ratio, chi, max_depth - 1, value)
        root.add_child(split_root)
    
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return root

In [156]:
# python supports passing a function as an argument to another function.
tree_gini = build_tree(data=X_train, impurity=calc_gini) # gini and goodness of split
#tree_entropy = build_tree(data=X_train, impurity=calc_entropy) # entropy and goodness of split
#tree_entropy_gain_ratio = build_tree(data=X_train, impurity=calc_entropy, gain_ratio=True) # entropy and gain ratio

In [157]:
tree_gini.recursive_print()

 [ROOT, feature=4]
	 [4=a, feature=2]: [{'e': 273, 'p': 31}]
		 [2=n, feature=8]: [{'e': 33, 'p': 5}]
			 [8=n, feature=19]: [{'e': 12, 'p': 3}]
				 [19=s, feature=0]: [{'e': 4, 'p': 3}]
					 [0=f, feature=18]: [{'e': 2, 'p': 1}]
						 [18=k, leaf]: [{'e': 1}]
						 [18=n, feature=20]: [{'e': 1, 'p': 1}]
							 [20=g, leaf]: [{'e': 1}]
							 [20=p, leaf]: [{'p': 1}]
					 [0=x, leaf]: [{'e': 2, 'p': 2}]
				 [19=y, leaf]: [{'e': 8}]
			 [8=p, feature=0]: [{'e': 9, 'p': 1}]
				 [0=f, feature=19]: [{'e': 2, 'p': 1}]
					 [19=s, leaf]: [{'e': 2}]
					 [19=y, leaf]: [{'p': 1}]
				 [0=x, leaf]: [{'e': 7}]
			 [8=w, feature=18]: [{'e': 12, 'p': 1}]
				 [18=k, feature=0]: [{'e': 5, 'p': 1}]
					 [0=f, leaf]: [{'e': 3}]
					 [0=x, feature=20]: [{'e': 2, 'p': 1}]
						 [20=g, leaf]: [{'p': 1}]
						 [20=p, leaf]: [{'e': 2}]
				 [18=n, leaf]: [{'e': 7}]
		 [2=w, feature=8]: [{'e': 107, 'p': 7}]
			 [8=g, feature=0]: [{'e': 22, 'p': 1}]
				 [0=b, feature=18]: [{'e': 9, 'p': 1}]


						 [2=e, leaf]: [{'e': 6}]
						 [2=g, leaf]: [{'e': 6}]
						 [2=n, feature=0]: [{'e': 5, 'p': 1}]
							 [0=f, leaf]: [{'e': 2, 'p': 1}]
							 [0=x, leaf]: [{'e': 3}]
				 [13=p, feature=1]: [{'e': 54, 'p': 4}]
					 [1=f, feature=12]: [{'e': 26, 'p': 3}]
						 [12=g, feature=2]: [{'e': 8, 'p': 1}]
							 [2=e, leaf]: [{'e': 4}]
							 [2=g, leaf]: [{'e': 3}]
							 [2=n, leaf]: [{'e': 1, 'p': 1}]
						 [12=p, feature=2]: [{'e': 7, 'p': 2}]
							 [2=e, leaf]: [{'e': 1, 'p': 2}]
							 [2=g, leaf]: [{'e': 3}]
							 [2=n, leaf]: [{'e': 3}]
						 [12=w, leaf]: [{'e': 11}]
					 [1=y, feature=2]: [{'e': 28, 'p': 1}]
						 [2=e, leaf]: [{'e': 11}]
						 [2=g, feature=12]: [{'e': 8, 'p': 1}]
							 [12=g, leaf]: [{'e': 2}]
							 [12=p, leaf]: [{'e': 3}]
							 [12=w, leaf]: [{'e': 3, 'p': 1}]
						 [2=n, leaf]: [{'e': 9}]
				 [13=w, feature=2]: [{'e': 47, 'p': 2}]
					 [2=e, leaf]: [{'e': 16}]
					 [2=g, feature=12]: [{'e': 15, 'p': 1}]
						 [12=g, leaf]: [{

## Tree evaluation

Complete the functions `predict` and `calc_accuracy`. (10 points)

In [175]:
def predict(node, instance):
    """
    Predict a given instance using the decision tree
 
    Input:
    - root: the root of the decision tree.
    - instance: an row vector from the dataset. Note that the last element 
                of this vector is the label of the instance.
 
    Output: the prediction of the instance.
    """
    pred = None
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    pass
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return node.pred

In [176]:
def calc_accuracy(node, dataset):
    """
    Predict a given dataset using the decision tree
 
    Input:
    - node: a node in the decision tree.
    - dataset: the dataset on which the accuracy is evaluated
 
    Output: the accuracy of the decision tree on the given dataset (%).
    """
    accuracy = 0
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    pass
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return accuracy 

After building the three trees using the training set, you should calculate the accuracy on the test set. For each tree print the training and test accuracy. Select the tree that gave you the best test accuracy. For the rest of the exercise, use that tree (when you asked to build another tree use the same impurity function and same gain_ratio flag). 

In [177]:
#### Your code here ####

## Post pruning

Iterate over all nodes in the tree that have at least a single child which is a leaf. For each such node, replace it with its most popular class. Calculate the accuracy on the testing dataset, pick the node that results in the highest testing accuracy and permanently change it in the tree. Repeat this process until you are left with a single node in the tree (the root). Finally, create a plot of the training and testing accuracies as a function of the number of nodes in the tree. (15 points)

In [178]:
#### Your code here ####

## Chi square pre-pruning

Consider the following p-value cut-off values: [1 (no pruning), 0.5, 0.25, 0.1, 0.05, 0.0001 (max pruning)]. For each value, construct a tree and prune it according to the cut-off value. Next, calculate the training and testing accuracy. On a single plot, draw the training and testing accuracy as a function of the tuple (p-value, tree depth). Mark the best result on the graph with red circle. (15 points)

In [179]:
### Chi square table values ###
# The first key is the degree of freedom 
# The second key is the p-value cut-off
# The values are the chi-statistic that you need to use in the pruning
chi_table = {1: {0.5 : 0.45,
                 0.25 : 1.32,
                 0.1 : 2.71,
                 0.05 : 3.84,
                 0.0001 : 100000},
             2: {0.5 : 1.39,
                 0.25 : 2.77,
                 0.1 : 4.60,
                 0.05 : 5.99,
                 0.0001 : 100000},
             3: {0.5 : 2.37,
                 0.25 : 4.11,
                 0.1 : 6.25,
                 0.05 : 7.82,
                 0.0001 : 100000},
             4: {0.5 : 3.36,
                 0.25 : 5.38,
                 0.1 : 7.78,
                 0.05 : 9.49,
                 0.0001 : 100000},
             5: {0.5 : 4.35,
                 0.25 : 6.63,
                 0.1 : 9.24,
                 0.05 : 11.07,
                 0.0001 : 100000},
             6: {0.5 : 5.35,
                 0.25 : 7.84,
                 0.1 : 10.64,
                 0.05 : 12.59,
                 0.0001 : 100000},
             7: {0.5 : 6.35,
                 0.25 : 9.04,
                 0.1 : 12.01,
                 0.05 : 14.07,
                 0.0001 : 100000},
             8: {0.5 : 7.34,
                 0.25 : 10.22,
                 0.1 : 13.36,
                 0.05 : 15.51,
                 0.0001 : 100000},
             9: {0.5 : 8.34,
                 0.25 : 11.39,
                 0.1 : 14.68,
                 0.05 : 16.92,
                 0.0001 : 100000},
             10: {0.5 : 9.34,
                  0.25 : 12.55,
                  0.1 : 15.99,
                  0.05 : 18.31,
                  0.0001 : 100000},
             11: {0.5 : 10.34,
                  0.25 : 13.7,
                  0.1 : 17.27,
                  0.05 : 19.68,
                  0.0001 : 100000}}

In [180]:
#### Your code here ####

Build the best 2 trees:
1. tree_max_depth - the best tree according to max_depth pruning
1. tree_chi - the best tree according to chi square pruning

In [181]:
#### Your code here ####

## Number of Nodes

Of the two trees above we will choose the one with fewer nodes. Complete the function counts_nodes and print the number of nodes in each tree. (5 points) 

In [182]:
def count_nodes(node):
    """
    Count the number of node in a given tree
 
    Input:
    - node: a node in the decision tree.
 
    Output: the number of node in the tree.
    """
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    return 1 + sum(count_nodes(child_node) for child_node in node.children)
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    

## Print the tree

Complete the function `print_tree` and execute it on your chosen tree. Your tree should be visualized clearly. You can use the following example as a reference:
```
[ROOT, feature=X0],
  [X0=a, feature=X2]
    [X2=c, leaf]: [{1.0: 10}]
    [X2=d, leaf]: [{0.0: 10}]
  [X0=y, feature=X5], 
    [X5=a, leaf]: [{1.0: 5}]
    [X5=s, leaf]: [{0.0: 10}]
  [X0=e, leaf]: [{0.0: 25, 1.0: 50}]
```
In each brackets:
* The first argument is the parent feature with the value that led to current node
* The second argument is the selected feature of the current node
* If the current node is a leaf, you need to print also the labels and their counts

(5 points)

In [183]:
# you can change the function signeture
def print_tree(node):
    '''
    prints the tree according to the example above

    Input:
    - node: a node in the decision tree

    This function has no return value
    '''
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    #This method is just an alias
    node.recursive_print(tab="  ")
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################